<a href="https://colab.research.google.com/github/emmanueluzodike/Natural_Language_Processing/blob/main/Chatbot/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instructions on running chat bot

To run the chatbot, you will need an apikey to be able to connect to OpenAI.

Follow the below instructions to get your apikey:

1. Sign up for an OpenAI account (if you already have an account, skip to step 3):
Visit https://beta.openai.com/signup/ and sign up for an account using your email address and a strong password.

2. Verify your email:
After signing up, check your email inbox for a verification email from OpenAI. Click on the link provided in the email to verify your account.


3. Log into your OpenAI account and Access your API key:
Log in to your account at https://beta.openai.com/login/. Navigate to the "API Keys" tab or visit https://beta.openai.com/account/api-keys/. Here, you will find your API key, which should be a long string of letters and numbers.

4. Keep your API key secure:
Treat your API key like a password; do not share it with anyone, and avoid uploading it to public repositories or websites. If you suspect your API key has been compromised, you can regenerate a new key or revoke access from the API keys page.

# Library Imports

In [ ]:
openai.api_key = "<API-KEY>" # Enter API key here

In [ ]:
# Can comment out after single execution in same runtime

## Download / install that take a long time
import nltk
#nltk.download('popular')
#!pip install eng-spacysentiment # Uncomment this on first run

## Python libraries
from collections import defaultdict
import string
import pickle
import random

## Preprocessing
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
import re

# Prretrained NLP model
#!pip install spacy # Uncomment this on first run
#!python -m spacy download en_core_web_sm # Uncomment this on first run
import spacy
from spacy import displacy
import eng_spacysentiment


# Create objects to be 
wnl = WordNetLemmatizer()
stopwords = stopwords.words('english')
nlp = spacy.load("en_core_web_sm")
sentiment = eng_spacysentiment.load()

!pip install openpyxl

# Prepping Knowledge Base

In [ ]:
import io
import requests


url = "https://github.com/emmanueluzodike/Natural_Language_Processing/blob/main/Chatbot/data/ai_news.xlsx?raw=true"

response = requests.get(url)
assert response.status_code == 200, f"Failed to fetch file, status code: {response.status_code}"

content = io.BytesIO(response.content)
df = pd.read_excel(content, engine='openpyxl')

#df = df.set_index(["Prompt", "completion"])
# Fill NaN values with empty string
df.fillna('', inplace=True)

df['content'] = df.Prompt +  " ." + df.completion
df['completion'] = df['completion'].str.replace('\n', ' ')
df['content'] = df['content'].str.replace('\n', ' ')
df['Prompt'] = df['Prompt'].str.replace('\n', ' ')
df["tokens"] = df["content"].apply(lambda x: len(x.split()))
#df = df.set_index(["Prompt", "completion"])
print(f"{len(df)} rows in the data.")
df.sample(5)

In [ ]:
# df = pd.read_excel('/content/ai_news.xlsx') #<----------Enter file path to xlsx file
# #df = df.set_index(["Prompt", "completion"])
# # Fill NaN values with empty string
# df.fillna('', inplace=True)

# df['content'] = df.Prompt +  " ." + df.completion
# df['completion'] = df['completion'].str.replace('\n', ' ')
# df['content'] = df['content'].str.replace('\n', ' ')
# df['Prompt'] = df['Prompt'].str.replace('\n', ' ')
# df["tokens"] = df["content"].apply(lambda x: len(x.split()))
# #df = df.set_index(["Prompt", "completion"])
# print(f"{len(df)} rows in the data.")
# df.sample(5)

100 rows in the data.


,Prompt,completion,content,tokens
40,Bard gets a little smarter as Google adds some...,No offers foundGoogle's new AI should better u...,Bard gets a little smarter as Google adds some...,534
21,China's Alibaba invites businesses to trial AI...,"SHANGHAI, April 7 (Reuters) - Tech giant Aliba...",China's Alibaba invites businesses to trial AI...,473
83,Read the memo Google CFO Ruth Porat sent to st...,Jump to Google is continuing its effort to cut...,Read the memo Google CFO Ruth Porat sent to st...,205
64,Alibaba's ChatGPT Rival to Offer Chinese and E...,"Your guide to a better future""We are at a tech...",Alibaba's ChatGPT Rival to Offer Chinese and E...,348
8,It’s Way Too Easy to Get Google’s Bard Chatbot...,Vittoria ElliottWhen Google announced the laun...,It’s Way Too Easy to Get Google’s Bard Chatbot...,949


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
df

In [ ]:
df['content'] = df['content'].str.strip()

In [ ]:
docs = []

# iterate through each row of the DataFrame
for index, row in df.iterrows():
  # tokenize the `content` coulmn using NLTK's word_tokenize() function
  tokens = nltk.word_tokenize(row['content'])
  # append the list of tokens to the `docs` list
  docs.append(tokens)

In [ ]:
print(docs)

In [ ]:
# preprocesses: remove non-alpha, remove stopwords, lemmatize
docs_preprocessed = [[wnl.lemmatize(w) for w in doc if w not in stopwords and w.isalpha()] for doc in docs]

In [ ]:
# create vocab set
vocab = set()
for doc in docs_preprocessed:
  doc_set = set(doc)
  vocab = vocab.union(doc_set)

vocab = sorted(list(vocab))
print('vocab length:', len(vocab))
vocab[:10]

In [ ]:
# creating vector
vectors = []
for doc in docs_preprocessed:
    vec = [doc.count(t) for t in vocab]
    vectors.append(vec)
print(vectors[0][:10])

In [ ]:
# function to compute cosine similarity
def cos_sim(v1, v2):
    return float(dot(v1, v2)) / (norm(v1) * norm(v2))

In [ ]:
# compute cosine similarity for the first doc, paired with all docs
for i, vec in enumerate(vectors):
    print('cosine similarity anat1 and vector', i+1, '=', format(cos_sim(vectors[0], vec), '.2f'))

In [ ]:
print(docs_preprocessed)

In [ ]:
print(vectors[0])

Find cosine similarity given a question

In [ ]:
# sample question
question = "How did Tesla's Model 3 perform in terms of sales?"

# preprocess the question
question_tokens = nltk.word_tokenize(question)
question_preprocessed = [wnl.lemmatize(w) for w in question_tokens if w not in stopwords and w.isalpha()]

# create a vector for the question
question_vector = [question_preprocessed.count(t) for t in vocab]

# compute cosine similarity between the question vector and all document vectors
for i, vec in enumerate(vectors):
    print('cosine similarity between question and vector', i+1, '=', format(cos_sim(question_vector, vec), '.2f'))

Prompt contsruction

In [ ]:
def construct_prompt(question, vectors):
  """
  Fetch relevant 
  """
  most_relevant_documents = get_relevant_documents(question, vectors)

  # header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
  header = """You are to take on the role of a chatbot. When a user asks a question, provide a response a conversational chat bot would and or Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""

  return header + " " + str(most_relevant_documents[0]) + " " + "\n\n Q: " + question + "\n A:"

In [ ]:
def get_relevant_documents(query, vectors):
  # compute cosine similarity for the query, paired with all docs
  query_vec = [wnl.lemmatize(w) for w in word_tokenize(query) if w not in stopwords and w.isalpha()]
  query_vec = [query_vec.count(t) for t in vocab]
  similarities = [(i, cos_sim(query_vec, vec)) for i, vec in enumerate(vectors)]

  # sort the similarities in descending order and select the top 3
  top_3 = sorted(similarities, key=lambda x: x[1], reverse=True)[:3]
  top_3_doc = []
  # print the top 3 document contents and their similarity scores
  for i, sim in top_3:
      #print(f"Document {i+1}: {df.loc[i, 'content']}\nSimilarity score: {sim:.2f}\n")
      content = df.loc[i, 'content'] 
      if len(content) > 4000:       # ensures prompt's token count does not exceed 4000
          content = content[:4000]
      top_3_doc.append(content)

  return top_3_doc

Give chatgpt prompt and context

In [ ]:
!pip install openai

In [ ]:
import openai

In [ ]:
# openai.api_key = "<API-KEY>"
COMPLETIONS_MODEL = "text-davinci-003"

In [ ]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [ ]:
def answer_query_with_context(
    query: str,
    vectors,
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        vectors,
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

# User Model Functionality

### Approach 1: Rule-based methods combined with sentiment analysis and POS tagging

In [ ]:
def user_registration1(): 
  # Prompt user
  new_user_introduction = input("Chatbot: What is your full name and date of birth (yyyymmdd)?\nUser: ")
  new_user_introduction = nlp(new_user_introduction)

  # Parse user input
  person_found = False
  dob_found = False
  for word in new_user_introduction.ents:
    if word.label_ == "PERSON":
      person_found = True
      name = word.text
    if word.label_ == "DATE":
      dob_found = True
      dob = word.text

  # Data validation
  while (person_found == False) or (not (' ' in name)):
    name = input("Chatbot: Full name not detected. Please capitalize and space between.\nUser: ")
    annotated_name = nlp(name)
    if annotated_name.ents[0].label_ == "PERSON":
      person_found = True
  while dob_found == False:
    dob = input("Chatbot: Date not detected. Please use format mmddyyyyy (no delimiters, pad left 0)\nUser: ")
    annotated_dob = nlp(dob)
    if annotated_dob.ents[0].label_ == "DATE":
      dob_found = True

  # Get more information
  """
    Test input:  I am a computer science student. I hate having to study. I have 3 siblings named Vincent, Andrew, and Emily. I like watching movies and to play games. I am a big fan of the Cowboys.
  """
  more_info = input("Chatbot: Tell me more about yourself!\nUser: ")
  sentences = more_info.split('.')
  more_info = nlp(more_info)

  # Analyze sentiment
  sentiment_dict = defaultdict()
  for sentence in sentences:
    doc = sentiment(sentence)
#    print(sentence, doc.cats)
    attitude = '+' if doc.cats['negative'] < 0.4 else '-'
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    for word in sentence.split(' '):
      sentiment_dict[word] = attitude

  pos_of_interest = {'VB', 'VBG', 'NNP', 'NNPS', 'NN', 'NNS'}
  pos_dict = defaultdict(set)

  # Add to dictionary 
  for word in more_info:
    if word.tag_ in pos_of_interest:
      pos_dict[word.tag_].add((word.text, sentiment_dict[word.text]))

  # Save dictionary as user model 
  user_model_fname = name.replace(' ', '').lower() + dob + '.p'
  pickle.dump(pos_dict, open(user_model_fname, 'wb'))

  # Give user login
  userID = user_model_fname[:-2]
  convo_start = input("Chatbot: Nice to meet you! Your userID is " + userID + " for future conversations. What do you want to talk about today?\nUser: ")

  return convo_start



In [ ]:
def user_signIn1(): 
  userID = input("Chatbot: Welcome back! What is your userID?\nUser: ")
  user_model_fname = userID.lower() + ".p"
  try:
    pos_dict = pickle.load(open(user_model_fname, 'rb'))
  except:
    option = input("Chatbot: Could not find that username. Enter 'Y' to try again, 'N' to register ")
    while option != 'Y' and option != 'N':
      option = input("Chatbot: Unexpected response. Please enter 'Y' or 'N'.\nUser: ")
    # User is new, create user model
    if option == 'N':
      convo_start = user_registration2() # Chang HERE for alt version
      return convo_start
    if option == 'Y':
      convo_start = user_signIn1()
  else: 
    if len(pos_dict) > 0:
      pos_tuples = list(pos_dict.items())
      pos = random.randint(0, len(pos_tuples)-1)
      tag = pos_tuples[pos][0]
      pos_list = list(pos_tuples[pos][1])
      word = random.randint(0, len(pos_list)-1)
      word = pos_list[word]

      # Generate prompt based on POS and sentiment
      word_text = word[0]
      if tag == "VB" or tag == "VBG":
        if tag == "VB": 
          if word[0][-1] == "e":
            word_text = word[0][:-1] + "ing"
          else:
            word_text = word[0] + "ing"
        if word[1] == "+":
          prompt = "Do you still enjoy " + word_text + "?"
        else:
          prompt = "Are you still stuck " + word_text + "?"
      elif tag == "NNP":
        if word[1] == "+":
          prompt = "How is " + word_text + " doing? (Good as ever)?"
        else:
          prompt = "How is " + word_text + " doing? (Pesky as ever)?"
      elif tag == "NNPS":
        if word[1] == "+":
          prompt = "How are the " + word_text + " doing? (Good as ever)?"
        else:
          prompt = "How are the " + word_text + " doing? (Pesky as ever)?"
      elif tag == "NNS":
        if word[1] == "+":
          prompt = "Any good " + word_text + " lately?"
        else:
          prompt = "Any terrible " + word_text + " lately?"
      else:
        if word[1] == "+":
          prompt = "Tell me about an exciting " + word_text + "."
        else:
          prompt = "Tell me about a bad " + word_text + "."
    else:
      birthdate = ''.join(filter(str.isdigit, user_model_fname))
      prompt = 'I don\'t know much about you, but I know your birthdate is ' + birthdate + '. How is the weather today?'

    greeting = input("Chatbot: " + prompt + "\nUser: ")

    # Respond to greeting and jump into second part of conversation
    sentiment_greeting = sentiment(greeting)
    if sentiment_greeting.cats['positive'] > 0.4:
      convo_start = input("Chatbot: That's great! What would you like to talk about today?\nUser: ")
    else:
      convo_start = input("Chatbot: That's tough.. What would you like to talk about today?\nUser: ")

    return convo_start 

### Approach 2: Collect data more directly and let Chat-GPT generate elaborate reponse

In [ ]:
def user_registration2(): 
  # Prompt user
  new_user_introduction = input("Chatbot: What is your full name and date of birth (yyyymmdd)?\nUser: ")
  new_user_introduction = nlp(new_user_introduction)

  # Parse user input
  person_found = False
  dob_found = False
  for word in new_user_introduction.ents:
    if word.label_ == "PERSON":
      person_found = True
      name = word.text
    if word.label_ == "DATE":
      dob_found = True
      dob = word.text

  # Data validation
  while (person_found == False) or (not (' ' in name)):
    name = input("Chatbot: Full name not detected. Please capitalize and space between.\nUser: ")
    annotated_name = nlp(name)
    if annotated_name.ents[0].label_ == "PERSON":
      person_found = True
  while dob_found == False:
    dob = input("Chatbot: Date not detected. Please use format mmddyyyyy (no delimiters, pad left 0)\nUser: ")
    annotated_dob = nlp(dob)
    if annotated_dob.ents[0].label_ == "DATE":
      dob_found = True

  # Get more information
  """
    Test input:  I am a computer science student. I hate having to study. I have 3 siblings named Vincent, Andrew, and Emily. I like watching movies and to play games. I am a big fan of the Cowboys.
  """
  likes_set = set()
  likes = input("Chatbot: What are your likes and interests?\nUser: ")
  likes = nlp(likes)
  for i in likes.ents:
    if i.label_ in {'EVENT', 'GPE', 'PERSON', 'MONEY', 'FAC', 'LANGUAGE', 'NORP', 'WORK_OF_ART', 'PRODUCT'}:
      likes_set.add(i.text)
  span = likes[:]
  chunks = list(span.noun_chunks) # Make doc into iterable
  for chunk in chunks: 
    if chunk.text != 'I':
      likes_set.add(chunk.text)

  dislikes_set = set()
  dislikes = input("Chatbot: What are your dislikes?\nUser: ")
  dislikes = nlp(dislikes)
  for i in dislikes.ents:
    if i.label_ in {'EVENT', 'GPE', 'PERSON', 'MONEY', 'FAC', 'LANGUAGE', 'NORP', 'WORK_OF_ART', 'PRODUCT'}: 
      dislikes_set.add(i.text)

  span = dislikes[:]
  chunks = list(span.noun_chunks) # Make doc into iterable
  for chunk in chunks:
    if chunk.text != 'I':
      dislikes_set.add(chunk.text)
  

  # Convert sets to strings
  likes_str = ', '.join(likes_set)
  dislikes_str = ', '.join(dislikes_set)
  # Create dictionary
  user_dict = {}
  user_dict['Name: '] = name
  user_dict['DOB (yyyymmdd): '] = dob
  user_dict['Likes: '] =likes_str
  user_dict['Dislikes: '] = dislikes_str

  # Save dictionary as user model 
  user_model_fname = name.replace(' ', '').lower() + dob + '.p'
  pickle.dump(user_dict, open(user_model_fname, 'wb'))

  # Give user login
  userID = user_model_fname[:-2]
  convo_start = input("Chatbot: Nice to meet you! Your userID is " + userID + " for future conversations. What do you want to talk about today?")

  return convo_start

In [ ]:
# TODO: Implement something similar to answer_query_with_context()
# Param - string formatted like the following:
'''
    User: christinetrinh20001125
    Name: Christine Trinh
    DOB (yyyymmdd): 20001125
    Likes: Spanish, movies, Lucy, a fan, the Cowboys
    Dislikes: rude people, video games, Santa
'''
# Returns - string response from ChatGPT
def temp(user_info):
  header = """I want you to take on the role of a chatbot. The below text represents the user information like Their name, dob, Likes and dislikes.
              I will like to to generate a random question based on their likes or dislikes or any other question you may wish based on thier info. "\n\nUser info:\n"""

  prompt = header + user_info

  response = openai.Completion.create(
            prompt=prompt,
            **COMPLETIONS_API_PARAMS
        )
  


  return response["choices"][0]["text"].strip(" \n")

In [ ]:
def user_signIn2():
  userID = input("\nChatbot: Welcome back! What is your userID?\nUser: ")
  user_model_fname = userID.lower() + ".p"
  try:
    user_dict = pickle.load(open(user_model_fname, 'rb'))
  except:
    option = input("\nChatbot: Could not find that username. Enter 'Y' to try again, 'N' to register ")
    while option != 'Y' and option != 'N':
      option = input("\nChatbot: Unexpected response. Please enter 'Y' or 'N'.\nUser: ")
    # User is new, create user model
    if option == 'N':
      convo_start = user_registration2() # Chang HERE for alt version
      return convo_start
    if option == 'Y':
      convo_start = user_signIn2()
  else:
        # Report 
    print('----Pickle File Contents----')
    for k, v in user_dict.items():
      print(k, v)
    print('----------------------------')

    # Build String to send to Chat-GPT
    user_info = ''
    for t, v in user_dict.items():
      user_info = user_info + t + v + '\n'

    #print(user_info)
    chatbot_greeting = temp(user_info)
    user_greeting = input('\nChatbot: Welcome back, ' + user_dict['Name: '] + '. ' + chatbot_greeting[10:] + '\nUser: ')

    # Respond to greeting and jump into second part of conversation
    sentiment_greeting = sentiment(user_greeting)
    if sentiment_greeting.cats['positive'] >= 0.4:
      convo_start = input("\nChatbot: That's great! What would you like to talk about today?\nUser: ")
    else:
      convo_start = input("\nChatbot: That's tough.. What would you like to talk about today?\nUser: ")

    return convo_start



In [ ]:
def begin_conversation():
  # Get user logged in or registered
  new_user = input("\nChatbot: Hello, I am Chatbot. Have we spoken before? (Y/N)\nUser: ")
  while new_user != 'Y' and new_user != 'N':
    new_user = input("\nChatbot: Unexpected response. Please enter 'Y' or 'N'.\nUser: ")
  # User is new, create user model
  if new_user == 'N':
    convo_start = user_registration2() # Change HERE for alt approach
  if new_user == 'Y':
    convo_start = user_signIn2()

  return convo_start
      
  
        


### Driver Code

In [ ]:
if __name__ == "__main__":
  convo_start = begin_conversation()
# Note: Use a simple English name while registering
  while convo_start != "q":
    if convo_start == 'q':
      print('Chatbot: Cya later!')
      break
    response = answer_query_with_context(convo_start, vectors)
    convo_start = input('\nChatbot: ' + response + '\nUser: ')





Chatbot: Hello, I am Chatbot. Have we spoken before? (Y/N)
User: Y

Chatbot: Welcome back! What is your userID?
User: briancox2000041420000414
----Pickle File Contents----
Name:  Brian Cox 20000414
DOB (yyyymmdd):  20000414
Likes:  a fan, Lucy, Cowboys, the Cowboys, Spanish, movies
Dislikes:  video games, Santa, rude people
----------------------------

Chatbot: Welcome back, Brian Cox 20000414. What is your favorite movie, Brian?
User: I really like Spiderman 2. What about you?

Chatbot: That's tough.. What would you like to talk about today?
User: What are your thoughts about AI

Chatbot: I think AI has the potential to be a powerful tool for good, but it is important to ensure that it is used responsibly and ethically. We need to ensure that AI systems are transparent and accountable, and that they are developed with safety in mind. We also need to ensure that AI is not used to discriminate against people or to cause harm.
User: Are you aware of your existence?

Chatbot: I'm aware 

I like to watch movies. I am a fan of the Cowboys. I like to play with Lucy. I enjoy learning Spanish. \\
I do not like video games. Nor do I like rude people, nor Santa. \\
